In [1]:
!pip install pyspark py4j
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext(master="local",appName="DataFrames")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=9fb3b4ec14a5579b23c4630764f8bb0c54993411e7166093de52d1ef751e9e16
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType,StringType, FloatType,Row
sqlContext = SQLContext(sc)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


##Beneficios
* Permite procesar datos como tabla de base de datos
* Poseen estructuras y pueden ser creados como DF
* Mejor optimizacion dado su optimizador de consultas

In [3]:
PATH = "/content/drive/MyDrive/D_S/Spark/curso-apache-spark-platzi-master/files/"
%cd /content/drive/MyDrive/D_S/Spark/curso-apache-spark-platzi-master/files
!ls

/content/drive/MyDrive/D_S/Spark/curso-apache-spark-platzi-master/files
deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [4]:
# La ventaja de definir un esquema y usarlo para cargar un archivo es el control. 
# En un esquema puedes definir qué tipos de datos estás esperando para qué columnas,
# mientras que si lees de la otra forma el archivo Spark trata de inferir los tipos y el esquema por su cuenta
juegoSchema = StructType([
    StructField("juego_id",IntegerType(),False),
    StructField("year",StringType(),False),
    StructField("temporada",StringType(),False),
    StructField("ciudad",StringType(),False),
])

# el option header nos indica que tiene un header para que este no se considere en el schema
juegoDF = sqlContext.read.schema(juegoSchema).option('header','true').csv(PATH+"juegos.csv") 
# !! EL sqlContext se usa en versiones viejas de spark, ahora se debe utilizar el SparkSession

In [5]:
juegoDF.show()

+--------+-------------+---------+--------+
|juego_id|         year|temporada|  ciudad|
+--------+-------------+---------+--------+
|       1|  1896 Verano|     1896|  Verano|
|       2|  1900 Verano|     1900|  Verano|
|       3|  1904 Verano|     1904|  Verano|
|       4|  1906 Verano|     1906|  Verano|
|       5|  1908 Verano|     1908|  Verano|
|       6|  1912 Verano|     1912|  Verano|
|       7|  1920 Verano|     1920|  Verano|
|       8|1924 Invierno|     1924|Invierno|
|       9|  1924 Verano|     1924|  Verano|
|      10|1928 Invierno|     1928|Invierno|
|      11|  1928 Verano|     1928|  Verano|
|      12|1932 Invierno|     1932|Invierno|
|      13|  1932 Verano|     1932|  Verano|
|      14|1936 Invierno|     1936|Invierno|
|      15|  1936 Verano|     1936|  Verano|
|      16|1948 Invierno|     1948|Invierno|
|      17|  1948 Verano|     1948|  Verano|
|      18|1952 Invierno|     1952|Invierno|
|      19|  1952 Verano|     1952|  Verano|
|      20|1956 Invierno|     195

In [6]:
from pyspark.sql.functions import col,countDistinct
juegoDF.agg(*(countDistinct(col(c)).alias(c) for c in juegoDF.columns)).show()

+--------+----+---------+------+
|juego_id|year|temporada|ciudad|
+--------+----+---------+------+
|      51|  51|       35|     2|
+--------+----+---------+------+



In [7]:
juegoDF.select("year").distinct().count()

51

In [8]:
pets = (juegoDF.columns)
for pet in pets :
    a = juegoDF.select(pet).distinct().count()
    print(str(pet)+"->"+str(a))

juego_id->51
year->51
temporada->35
ciudad->2


In [10]:
juegoDF.select(*juegoDF.columns).distinct().count()

51